<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# Try out the NVIDIA Multimodal PDF Data Extraction Blueprint! 🤙

Welcome!

In this notebook, we will run inference on the new NVIDIA Multimodal PDF Data Extraction Blueprint. This blueprint uses a state-of-the-art multimodal model to extract data from PDFs, PowerPoints, and images. 


**Important Notes**: 
1. In order to run this notebook, you need to visit NGC and get a key for the Multimodal PDF Data Extraction Blueprint.
3. **Because this model uses a 2xA100 cluster, it can get expensive to leave on for a long time. If you're looking to host this workflow permanently, please reach out to the Brev team and we can chat!**

### Help us make this tutorial better! Please provide feedback on the [Discord channel](https://discord.gg/y9428NwTh3) or on [X](https://x.com/brevdev).

# Introduction

NVIDIA-Ingest is a scalable, performance-oriented document content and metadata extraction microservice. Including support for parsing PDFs, Word and PowerPoint documents, it uses specialized NVIDIA NIM microservices to find, contextualize, and extract text, tables, charts and images for use in downstream generative applications.

NVIDIA Ingest enables parallelization of the process of splitting documents into pages where contents are classified (as tables, charts, images, text), extracted into discrete content, and further contextualized via optical character recognition (OCR) into a well defined JSON schema. From there, NVIDIA Ingest can optionally manage computation of embeddings for the extracted content, and also optionally manage storing into a vector database Milvus.

# Architecture

include that literally unreadable goofy picture here

# Getting started

1. create a python function where user can provide the 3 inputs and it create the .env
2. Provide the command that a user should run in the terminal to docker compose
3. Ensure the relevant outputs are up
4. Show 2-3 examples of the ingestion 